In [7]:
from film_review_explorer import TextProcessor

ImportError: cannot import name 'TextProcessor' from 'film_review_explorer' (/Users/pipchang/opt/anaconda3/envs/pipjenew/lib/python3.10/site-packages/film_review_explorer/__init__.py)

- key words

- Word cloud

- Word in context, word trend

- nice plots with plotly

